<a href="https://colab.research.google.com/github/SYWoo02/KNU_Capstone_Project_2025/blob/main/DeepDDI2%EB%A1%9C%EB%B6%80%ED%84%B0_%EC%83%9D%EC%84%B1%ED%95%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%86%8C%EA%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepDDI2의 약물 범위를 '성인병(고혈압, 당뇨병, 고지혈증, 심혈관 질환)'으로 축소
* drugbank_id: DrugBank 내 약물 고유번호
* name: 약물 이름 (영문)
* type: 약물의 분자 유형
* groups: 승인 상태
* categories: (약리학적 분류) 약물이 어떤 치료영역에 쓰이는지, 작용기전이 어떤지 파악 가능
* SMILES: 분자의 화학 구조를 문자열로 표현
* PubChem_CID: 승인 상태
* disease_groups: 어느 성인병 치료에 쓰이는 약물인지?

In [5]:
import pandas as pd
data1 = pd.read_csv("성인병 관련 약물 _ 질환별 분류 _ 결측치 제거.csv")
data1.head(5)

,drugbank_id,name,type,groups,categories,SMILES,PubChem_CID,disease_groups
0,DB00035,Desmopressin,small molecule,approved,"Agents that produce hypertension|Amino Acids, ...",NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,NaN,"['고혈압', '당뇨병']"
1,DB00050,Cetrorelix,small molecule,approved|investigational,"Amino Acids, Peptides, and Proteins|Anti-Gonad...",CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...,25074887.0,['당뇨병']
2,DB00104,Octreotide,small molecule,approved|investigational,"Amino Acids, Peptides, and Proteins|Antineopla...",[H][C@]1(NC(=O)[C@H](CCCCN)NC(=O)[C@@H](CC2=CN...,448601.0,['당뇨병']
3,DB00175,Pravastatin,small molecule,approved,Agents Causing Muscle Toxicity|Anticholesterem...,[H][C@]12[C@H](C[C@H](O)C=C1C=C[C@H](C)[C@@H]2...,54687.0,['고지혈증']
4,DB00177,Valsartan,small molecule,approved|investigational,Agents Acting on the Renin-Angiotensin System|...,CCCCC(=O)N(CC1=CC=C(C=C1)C1=CC=CC=C1C1=NNN=N1)...,60846.0,['고혈압']


# 성인병 약물 조합과 상호작용 라벨 (1~113)
### DrugBank 내 존재하는 조합들 (DeepDDI2 데이터에서 제공된 조합 데이터)로만 구성

### 성인병 관련 약물조합 w interaction_type(label)_35.csv : 가능한 모든 약물조합의 상호작용 데이터 (포괄적)

In [107]:
# 데이터 불러오기
ddi_data = pd.read_csv('DrugBank_known_ddi.csv')
combo_data = pd.read_csv('성인병 관련 약물 조합.csv')

# 컬럼명 맞춰주기 (ddi_data)
ddi_data = ddi_data.rename(columns={'drug1': 'Drug1_ID', 'drug2': 'Drug2_ID', 'Label': 'Label'})

# Key 만들기 (약물쌍 순서 정렬)
ddi_data['combo_key'] = ddi_data.apply(lambda row: tuple(sorted([row['Drug1_ID'], row['Drug2_ID']])), axis=1)
combo_data['combo_key'] = combo_data.apply(lambda row: tuple(sorted([row['Drug1_ID'], row['Drug2_ID']])), axis=1)

# DrugBank_known_ddi에서 combo_key, label만 추출
ddi_labels = ddi_data[['combo_key', 'Label']]

# 성인병 약물조합에 label 붙이기 (left join)
combo_data = combo_data.merge(ddi_labels, on='combo_key', how='left')

# 필요하면 combo_key 제거
combo_data.drop(columns=['combo_key'], inplace=True)

#combo_data

# 결측치 (상호작용 X) 제거, 최종 결과 출력
# 113가지 상호작용 중 35가지만, 총 11,233개의 행
combo_data2 = combo_data.dropna()
combo_data2['Label'] = combo_data2['Label'].astype(int)
print(combo_data2)

# csv 생성
combo_data2.to_csv('성인병 관련 약물조합 w interaction_type(label)_35.csv', index=False, encoding='utf-8-sig')

      Disease1 Drug1_ID   Drug1_Name Disease2 Drug2_ID     Drug2_Name  Label
1         고지혈증  DB00175  Pravastatin      고혈압  DB00177      Valsartan     89
11        고지혈증  DB00175  Pravastatin      고혈압  DB00264     Metoprolol      3
12        고지혈증  DB00175  Pravastatin      고혈압  DB00270     Isradipine      3
19        고지혈증  DB00175  Pravastatin      고혈압  DB00343      Diltiazem      3
24        고지혈증  DB00175  Pravastatin      고혈압  DB00381     Amlodipine      3
...        ...      ...          ...      ...      ...            ...    ...
74379     심혈관계  DB09236   Lacidipine      당뇨병  DB08971  Fluocortolone     20
74384     심혈관계  DB09236   Lacidipine      당뇨병  DB11921    Deflazacort     20
74422     심혈관계  DB09238   Manidipine      당뇨병  DB02638   Terlipressin     75
74427     심혈관계  DB09238   Manidipine      당뇨병  DB06292  Dapagliflozin     89
74593     심혈관계  DB11362    Selexipag      당뇨병  DB02638   Terlipressin     75

[11233 rows x 7 columns]


In [111]:
print(combo_data2['Label'].value_counts())
# 데이터 불균형 심함

Label
75     5480
113    2060
89      711
3       577
108     451
57      283
47      277
95      235
5       164
109     158
60      138
74      104
12       92
20       87
55       72
10       59
1        40
13       40
63       37
99       32
81       25
102      24
80       20
49       18
104       8
54        8
28        8
92        8
53        5
65        4
7         2
98        2
58        2
87        1
21        1
Name: count, dtype: int64


In [112]:
combo_data2_label_list = combo_data2['Label'].unique()
print(combo_data2_label_list)
# [89, 3, 104, 20, 113, 75, 108, 12, 47, 102, 65, 63, 13, 55, 74, 7, 95, 98, 28, 99, 57,
#   1, 10, 109, 60, 49, 5, 54, 80, 81, 53, 92, 87, 21, 58]; 35가지

[ 89   3 104  20 113  75 108  12  47 102  65  63  13  55  74   7  95  98
  28  99  57   1  10 109  60  49   5  54  80  81  53  92  87  21  58]


### 성인병 관련 약물조합 w interaction_type(label)_26.csv : 실제로 의미 있는 상호작용만 추려낸 데이터
* 의미 있는 상호작용 유형: 혈중 농도/대사 변화/흡수-배설 방해/심혈관 위험

In [115]:
data2 = pd.read_csv("성인병 관련 약물조합 w interaction_type(label)_26.csv")
data2 = data2.drop('Unnamed: 0',axis=1)
data2.tail(5)
# 113가지 상호작용 중 26가지만, 총 5219개의 행

,Disease1,Drug1_ID,Drug1_Name,Disease2,Drug2_ID,Drug2_Name,Label
5214,고혈압,DB09242,Moxonidine,당뇨병,DB08907,Canagliflozin,113
5215,고혈압,DB09477,Enalaprilat,당뇨병,DB06292,Dapagliflozin,113
5216,고혈압,DB09477,Enalaprilat,당뇨병,DB08907,Canagliflozin,102
5217,고혈압,DB11738,Rilmenidine,당뇨병,DB02638,Terlipressin,75
5218,고혈압,DB12945,Dihydralazine,당뇨병,DB02638,Terlipressin,75


In [116]:
print(data2['Label'].value_counts())
# 데이터 불균형 심함

Label
75     2619
113     761
3       350
108     284
89      266
47      193
57      176
5       142
60       81
109      64
12       60
20       43
55       35
10       29
1        18
81       16
102      16
63       14
74       14
49       10
80       10
28        8
54        4
65        3
53        2
98        1
Name: count, dtype: int64


In [117]:
data2_label_list = data2['Label'].unique()
print(data2_label_list)
# [89, 3, 113, 75, 12, 102, 65, 63, 47, 20, 55, 74, 108,
#  98, 28, 10, 60, 49, 5, 54, 80, 81, 57, 1, 109, 53] ; 26가지

[ 89   3 113  75  12 102  65  63  47  20  55  74 108  98  28  10  60  49
   5  54  80  81  57   1 109  53]


In [118]:
# 성인병 약물 조합 내 나타난 상호작용들만 추출
li_list = pd.read_csv("Interaction_information(최종).csv")

# 추출할 InteractionType 목록
target = [89, 3, 113, 75, 12, 102, 65, 63, 47, 20, 55, 74, 108, 98, 28, 10, 60, 49, 5, 54, 80, 81, 57, 1, 109, 53]
target = [str(x) for x in target]  # 문자열로 변환

# InteractionType 컬럼이 문자열일 수 있으므로 통일
li_list['InteractionType'] = li_list['InteractionType'].astype(str)

# 원하는 InteractionType만 필터링
data2_li_list = li_list[li_list['InteractionType'].isin(target)]

# CSV로 저장
# data2_li_list.to_csv('성인병 관련 약물조합 내 상호작용 추출.csv', index=False, encoding='utf-8-sig')

# 결과 출력
data2_li_list

,Subject,InteractionType,Description,DDIType
0,3,1,The risk or severity of congestive heart failu...,Type_56
2,2,3,The serum concentration of #Drug2 can be incre...,Type_52
4,2,5,#Drug2 may increase the atrioventricular block...,Type_7
9,2,10,The therapeutic efficacy of #Drug2 can be incr...,Type_181
11,2,12,#Drug1 may increase the hyperkalemic activitie...,Type_188
19,2,20,The serum concentration of #Drug1 can be decre...,Type_48
27,3,28,The risk or severity of rhabdomyolysis can be ...,Type_197
46,2,47,#Drug2 may increase the antihypertensive activ...,Type_68
48,2,49,#Drug2 may decrease the bronchodilatory activi...,Type_67
52,2,53,#Drug1 may increase the QTc-prolonging activit...,Type_116


# 성인병 관련 약물의 위험 상호작용 유형 (9가지)
### DeepDDI2의 conflicting_types(15가지) 데이터 기반

| InteractionType | 한글 설명                                            |
| --------------- | ------------------------------------------------ |
| **3**           | #Drug1과 함께 복용할 때 #Drug2의 혈중 농도가 증가할 수 있음         |
| **6**           | #Drug2와 함께 복용할 때 #Drug1의 활성 대사물의 혈중 농도가 증가할 수 있음 |
| **13**          | #Drug1이 #Drug2의 흡수를 감소시킬 수 있음                    |
| **20**          | #Drug2와 함께 복용할 때 #Drug1의 혈중 농도가 감소할 수 있음         |
| **33**          | #Drug1과 함께 복용할 때 #Drug2의 활성 대사물의 혈중 농도가 감소할 수 있음 |
| **48**          | #Drug2와 함께 복용할 때 #Drug1의 흡수가 감소할 수 있음            |
| **56**          | #Drug2가 #Drug1의 흡수를 증가시킬 수 있음                    |
| **89**          | #Drug1과 함께 복용할 때 #Drug2의 대사가 감소할 수 있음            |
| **104**         | #Drug2와 함께 복용할 때 #Drug1의 대사가 증가할 수 있음            |


In [54]:
data4 = pd.read_csv("위험 약물 상호작용 (9가지).csv")
print(data4['InteractionType'].unique())
#[3, 6, 13, 20, 33, 48, 56, 89, 104]

data4

[  3   6  13  20  33  48  56  89 104]


,Subject,InteractionType,Description,DDIType
0,2,3,The serum concentration of #Drug2 can be incre...,Type_52
1,2,6,The serum concentration of the active metaboli...,Type_214
2,2,13,#Drug1 can cause a decrease in the absorption ...,Type_189
3,2,20,The serum concentration of #Drug1 can be decre...,Type_48
4,2,33,The serum concentration of the active metaboli...,Type_76
5,2,48,The absorption of #Drug1 can be decreased when...,Type_66
6,2,56,#Drug2 can cause an increase in the absorption...,Type_110
7,2,89,The metabolism of #Drug2 can be decreased when...,Type_139
8,2,104,The metabolism of #Drug1 can be increased when...,Type_85


In [123]:
# 데이터 로드
conflicting_types = pd.read_csv('conflicting_types.csv')
conflicting_types = conflicting_types['type']

# 위험 상호작용만 추출
dangerous_combos = combo_data2[combo_data2['Label'].isin(conflicting_types)]

# CSV로 저장
dangerous_combos.to_csv('성인병 관련 약물조합 (_35) 내 위험 상호작용 추출.csv', index=False, encoding='utf-8-sig')

# 결과 출력
print(dangerous_combos['Label'].value_counts())
dangerous_combos

Label
89     711
3      577
20      87
13      40
104      8
Name: count, dtype: int64


,Disease1,Drug1_ID,Drug1_Name,Disease2,Drug2_ID,Drug2_Name,Label
1,고지혈증,DB00175,Pravastatin,고혈압,DB00177,Valsartan,89
11,고지혈증,DB00175,Pravastatin,고혈압,DB00264,Metoprolol,3
12,고지혈증,DB00175,Pravastatin,고혈압,DB00270,Isradipine,3
19,고지혈증,DB00175,Pravastatin,고혈압,DB00343,Diltiazem,3
24,고지혈증,DB00175,Pravastatin,고혈압,DB00381,Amlodipine,3
...,...,...,...,...,...,...,...
74361,심혈관계,DB09236,Lacidipine,당뇨병,DB01234,Dexamethasone,20
74364,심혈관계,DB09236,Lacidipine,당뇨병,DB01384,Paramethasone,20
74379,심혈관계,DB09236,Lacidipine,당뇨병,DB08971,Fluocortolone,20
74384,심혈관계,DB09236,Lacidipine,당뇨병,DB11921,Deflazacort,20


# 성인병 관련 약물들 간 유사 약물 파악
### DeepDDI2의 drug_similarity 기반
### 후에 위험 상호작용이 있는 약물들을 (위험 상호작용이 없는) 유사 약물로 대체하고자 함.

In [27]:
data3 = pd.read_csv("성인병_유사_약물_비교.csv",encoding='euc-kr')
data3.head(5)

,기존_약물_ID,기존_약물_이름,대체가능_약물_ID,대체가능_약물_이름
0,DB00389,Carbimazole,DB00550,Propylthiouracil
1,DB00389,Carbimazole,DB01086,Benzocaine
2,DB00389,Carbimazole,DB00201,Caffeine
3,DB00389,Carbimazole,DB00779,Nalidixic Acid
4,DB00389,Carbimazole,DB00292,Etomidate


## 위험 상호작용이 있는 약물을 유사약물로 대체

In [149]:
# 데이터 불러오기
risk_combos = pd.read_csv('성인병 관련 약물조합 (_35) 내 위험 상호작용 추출.csv')  # 위험 상호작용
alternatives = pd.read_csv('성인병_유사_약물_비교.csv', encoding='euc-kr')  # 유사약물 매핑

# 대체약물 매핑 (딕셔너리)
alt_mapping2 = pd.Series(alternatives['대체가능_약물_ID'].values, index=alternatives['기존_약물_ID']).to_dict()

# 위험조합 데이터에서 A ↔ B로 순서 상관없이 정렬
risk_combo_set = set([
    tuple(sorted([str(row['Drug1_ID']), str(row['Drug2_ID'])]))
    for _, row in risk_combos.iterrows()
])

# 대체약물-상대약물 위험 여부 확인 함수 (여기서만 대체약물 대입)
def check_new_risk(row):
    drug1, drug2 = str(row['Drug1_ID']), str(row['Drug2_ID'])
    alt1 = str(alt_mapping2.get(drug1, '')) if alt_mapping2.get(drug1) else None
    alt2 = str(alt_mapping2.get(drug2, '')) if alt_mapping2.get(drug2) else None

    # 대체약물 열 추가
    row['Drug1_대체약물'] = alt1 if alt1 else ''
    row['Drug2_대체약물'] = alt2 if alt2 else ''

    # 대체약물의 위험 상호작용 검증
    result = []
    # Drug1을 대체약물로 교체 후 위험조합 확인
    if alt1:
        alt1_combo = tuple(sorted([alt1, drug2]))
        if alt1_combo in risk_combo_set:
            result.append(f"Drug1 대체({alt1})-Drug2 위험")

    # Drug2를 대체약물로 교체 후 위험조합 확인
    if alt2:
        alt2_combo = tuple(sorted([alt2, drug1]))
        if alt2_combo in risk_combo_set:
            result.append(f"Drug2 대체({alt2})-Drug1 위험")
    row['대체약물_검증결과'] = ', '.join(result) if result else "대체약물-상대약물 위험 없음"
    return row

# 다시 검증 결과 생성
risk_combos = risk_combos.apply(check_new_risk, axis=1)

# 결과 확인
risk_combos.head(10)

# CSV로 저장
# risk_combos.to_csv('성인병 위험약물 w 대체가능한 유사약물.csv', index=False, encoding='utf-8-sig')

# Drug1_대체약물 ↔ Drug2_대체약물 외에 Drug1 ↔ Drug2_대체약물, Drug2 ↔ Drug1_대체약물까지 모두 위험 상호작용으로 나타나는 케이스가 없음을 확인.

,Disease1,Drug1_ID,Drug1_Name,Disease2,Drug2_ID,Drug2_Name,Label,Drug1_대체약물,Drug2_대체약물,대체약물_검증결과
0,고지혈증,DB00175,Pravastatin,고혈압,DB00177,Valsartan,89,DB13025,DB12924,대체약물-상대약물 위험 없음
1,고지혈증,DB00175,Pravastatin,고혈압,DB00264,Metoprolol,3,DB13025,DB00313,대체약물-상대약물 위험 없음
2,고지혈증,DB00175,Pravastatin,고혈압,DB00270,Isradipine,3,DB13025,DB13783,대체약물-상대약물 위험 없음
3,고지혈증,DB00175,Pravastatin,고혈압,DB00343,Diltiazem,3,DB13025,DB00222,대체약물-상대약물 위험 없음
4,고지혈증,DB00175,Pravastatin,고혈압,DB00381,Amlodipine,3,DB13025,DB01051,대체약물-상대약물 위험 없음
5,고지혈증,DB00175,Pravastatin,고혈압,DB00401,Nisoldipine,3,DB13025,DB00786,대체약물-상대약물 위험 없음
6,고지혈증,DB00175,Pravastatin,고혈압,DB00528,Lercanidipine,3,DB13025,DB00561,대체약물-상대약물 위험 없음
7,고지혈증,DB00175,Pravastatin,고혈압,DB00559,Bosentan,104,DB13025,DB13783,대체약물-상대약물 위험 없음
8,고지혈증,DB00175,Pravastatin,고혈압,DB00622,Nicardipine,3,DB13025,DB09345,대체약물-상대약물 위험 없음
9,고지혈증,DB00175,Pravastatin,고혈압,DB00661,Verapamil,3,DB13025,DB00613,대체약물-상대약물 위험 없음
